In [1]:
import pandas as pd

%load_ext rpy2.ipython

In [2]:
df = pd.read_csv('../data_prep/bpd_prepped.csv')
df.head()

,name,department_name,title,regular,retro,other,overtime,injured,detail,quinn,total_earnings,zip,year,allegations,sustained,promotion
0,abascianojoseph,Boston Police Department,Police Officer,$68181.24,$0.00,$3763.50,$14177.65,$0.00,$4867.00,$0.00,90989.39,02132-3802,2011,2,0,0
1,abascianojoseph,Boston Police Department,Police Officer,$69432.97,$0.00,$800.00,$16767.14,$0.00,$1767.00,$0.00,88767.11,02132-3802,2012,0,0,0
2,abascianojoseph,Boston Police Department,Police Officer,$66673.45,$0.00,$800.00,$20318.50,$3163.30,$890.00,$0.00,91845.25,02132-3802,2013,1,0,0
3,abascianojoseph,Boston Police Department,Police Officer,$19469.90,$10200.36,$1491.24,$2565.64,$60753.78,$2553.00,$825.04,97858.96,02132,2014,0,0,0
4,abascianojoseph,Boston Police Department,Police Officer,$-886.20,$0.00,$8088.35,$544.83,$81996.78,$0.00,$20667.80,110411.56,02132,2015,0,0,0


In [3]:
verdicts = df.sustained.values
verdicts.shape

(25522,)

In [4]:
guilty_previously = [0] * len(verdicts)
current_guy = df['name'][0]
known_offender = 0
for i in range (1, len(guilty_previously)-1):
    if df['name'][i] == current_guy:

        if verdicts[i-1] == 1 or known_offender == 1:
            guilty_previously[i] = 1
            known_offender = 1
    else:
        current_guy = df['name'][i]
        known_offender = 0

In [5]:
df['known_offender'] = guilty_previously

In [6]:
known_off_df = df[df.known_offender == 1]


In [7]:
years = known_off_df.year.value_counts().sort_index().index
total = known_off_df.year.value_counts().sort_index().sum()
prom = known_off_df.promotion.sum()
rate = prom/total

known_off_graph = pd.DataFrame({'total': total, 'promoted': prom, 'rate': rate, 'known_offender': "Known Offenders"}, index =[0])

In [8]:
non_off_df = df[df.known_offender == 0]

In [9]:
years = non_off_df.year.value_counts().sort_index().index
total = non_off_df.year.value_counts().sort_index().sum()
prom = non_off_df.promotion.sum()
rate = prom/total

non_off_graph = pd.DataFrame({'total': total, 'promoted': prom, 'rate': rate, 'known_offender': "Non-offenders"}, index=[0])

In [10]:
non_off_graph

,total,promoted,rate,known_offender
0,23384,988,0.042251,Non-offenders


In [11]:
graph = pd.concat([non_off_graph, known_off_graph], ignore_index=True)

graph.head(30)


,total,promoted,rate,known_offender
0,23384,988,0.042251,Non-offenders
1,2138,90,0.042095,Known Offenders


In [12]:
%%R -i graph 

library(ggplot2)
library(ggthemes)
library(stringr)
ggplot(graph) +
aes(x=known_offender, y=rate, fill=factor(known_offender)) +
#facet_wrap(~known_offender, as.table=FALSE) +
scale_x_discrete(limits=rev) +
geom_bar(stat="identity", position="dodge") +
theme_solarized() +
labs(title=str_wrap("In the last 10 years, the BPD has promoted officers at a consistent rate, regardless of whether or not it has found them guilty of misconduct",40),x=element_blank(), y="Promotion Rate") +
guides(fill=element_blank()) +
scale_fill_brewer(palette="Dark2") +
theme(strip.text.x = element_blank(), legend.position="none")


ggsave("../output/known_offender.png", width=4, height=6)

